# NetCDF data with `xarray` in Python

In python , the `xarray` package is a very powerful library to manipulate multi-dimensional data, and it can read from NetCDF files. 

You can check the documentation here: https://docs.xarray.dev/en/stable/index.html.
The "getting started" documentation is a very good starting point, and the user guide is well written. Do not hesitate to reference it when you want to do more advanced things with `xarray` and to understand how it works. 

This notebook is structured as follows:
1. Introduction to xarray's DataArray and Dataset classes
2. Opening and exploring NetCDF files
3. Indexing and subsetting
4. Statistical operations
5. Grouping
6. Wrap-up on examples
7. Plotting

In [ ]:
# Import xarray
import xarray as xr

# Additionnal packages 
import numpy as np
import matplotlib.pyplot as plt

## Introduction to xarray's DataArray and Dataset classes

`xarray` is built on two fundamental classes: `DataArray` and `Dataset`, Datasets being collections of DataArrays.
Therefore, let us start by familiarizing ourselves with these two objects.

### DataArray
DataArrays are the fundamental bricks of xarray. They are objects that contains a variable along with its coordinates and attributes.

Here, we build a dummy dataset to show how it works. 

In [ ]:
# Generate a dummy time series
t = np.arange(0,10,0.2) # coordinate (e.g. time)
print("t:", t) # Content of t

s = np.sin(t) + np.random.random(len(t))/4 # data series: here a sine with some random noise
print("s:", s) # Content of s
print("Type of s:", type(s)) # Check the type of s: For the moment, s in a numpy array

plt.plot(t, s, marker = 'o') # Visualize s against t with matplotlib 

In [ ]:
# From this time series, we can create a DataArray, the fundamental xarray object
da = xr.DataArray(s, dims = ('t'), coords = {'t':t})
# Visualize the object: See it contains the s numpy array, associated with t as a coordinate, and no attributes.
da

In [ ]:
# You can access the values of the variable as a numpy array
da.values

In [ ]:
# Same for the values of the coordinates
da.t.values

In [ ]:
# A DataArray may also contain attributes (e.g. units)
da.attrs # Display attributes: For the moment, they are empty

In [ ]:
# But we can add a unit
da.attrs["units"] = "m"
print(da.attrs) # Display attributes: Now they include the unit
da # Visualise da: See that now the units are at the end the the attributes menu

### Dataset
Dataset are another very important class of objects in xarray. A Dataset is a container of DataArrays.

In [ ]:
# Let us create a Dataset that contains the DataArray that we just created.
ds = xr.Dataset({"s":da})
ds # Visualise the Dataset, you see the structure is the same as before, but a new category "Data variables" appears, that contains s

In [ ]:
# You can access the DataArray object with 
ds.s # or ds['s']

In [ ]:
# A Dataset is not really useful if it contains just one DataArray. Let us add other DataArray as variable in the dataset
# Task: Create another dummy time series that is a cosinus plus some random noise, and transform it into a DataArray
c = ... + np.random.random(len(t))/3 # c is the cosinus of t plus some noise
da2 = ... # Transform c into a DataArray
... # Visualize this new DataArray

In [ ]:
# Add the new DataArray as a variable in the Dataset
ds['c'] = da2
ds # Visualise ds: You see now there are two variables: s and c

### Built-in plot method
Last thing about xarray's DataArrays: They have a very useful built-in `.plot()` method. 
If the data is 1D, it will plot it as a line, if it is 2D, it will plot it as a map, and if it has 3 or more dimensions, it will show an histogram of all the values. 
We quickly show how it works here, so that we can use it in the following. More customizations will be introduced by the end of this notebook.

In [ ]:
# Quick plot of our first DataArray
da.plot()

In [ ]:
# Plot the two time series on top of each other
da.plot(label = "sinus") # Plot the sinus time series
da2.plot(label = "cosinus") # Task: Plot the cosinus time series
plt.legend()

In [ ]:
# Remember that you can access the DataArrays from the Dataset, therefore this code will yield the exact same result as before:
ds.s.plot(label = "sinus") # Plot the sinus time series
ds.c.plot(label = "cosinus") # Task: Plot the cosinus time series
plt.legend()

## Opening NetCDF files
Now that we have covered the fundamentals, let us use some real data. Opening a netcdf file with xarray is straightforward:

In [ ]:
# To open a NetCDF file, use the open_dataset function
file = "../../data_samples/netcdf/E-OBS/UK_monthly.nc" # Path to our netcdf file
ds = xr.open_dataset(file)

Now, visualise the object that was created and answer these questions:
* What variables are present in the file? What are their unit and range?
* Over which area and which period are these variables provided?
* What is the frequency and the spatial resolution of the data? 

In [ ]:
# Visualise the object that was created: It is a Dataset.
ds

In [ ]:
# Remember, each variable can be accessed to see its values, coordinates and attributes 
ds.tg

In [ ]:
# For the moment, the data is 3D, so if we try to plot it now, it will show an histogram of all the values. 
# This can be useful to check the range of values and make sure it makes sense. 
ds.tg.plot()
# Later on, we will se how to subset or aggregate the data to display maps or series

*-- Advanced --*
*When you open a dataset with xarray, the data is loaded in memory only once you need it.*
*However, you can force it to load with `ds.load()`*

### Indexing
Datasets and DataArrays both have a `.sel()` method to access the values of the variable(s) at specified coordinates (i.e. to subset the data). Alternatively, the `.isel()` method can be used with integer indices.

#### `.sel()` examples

In [ ]:
# Extract the values for the point of coordinates 50.625°N 1.625°E
ds_pt = ds.sel(latitude = 50.625, longitude = 1.625)
# Notice the object now has only one dimension: time
ds_pt

In [ ]:
# Because ds_pt is 1D, you can now plot its variables as series. Here is an example for the temperature
ds_pt.tg.plot()

In [ ]:
# Get the values for the month of January 2002
ds_Jan02 = ds.sel(time = "2002-01")
# Notice the object still has three dimension, but the time dimension is only 1.
ds_Jan02

In [ ]:
# We can threfore plot its variables as maps. Here is an example for precipitation
ds_Jan02.rr.plot()

In [ ]:
# Get the values of precipitation for the point of coordinates 50.625°N 1.625°E in January 2002
ds_pt_Jan02 = ds.rr.sel(time = "2002-01", latitude = 50.625, longitude = 1.625)
# Notice the object contains just that number
ds_pt_Jan02

#### Obtaining values for precise coordinates not provided

**NB: `.sel()` expects values of coordinates that are present in the dataset (i.e. if you ask for the longitude 0.0, which is not provided, it will raise an error).** 

In [ ]:
# Example that raises an error
ds.sel(longitude = 0.0)

There are two ways around it:
1. Obtain the nearest values by adding the `method = "nearest"` option in `.sel()`
2. Interpolate the values to the point you want

In [ ]:
# Using method = "nearest": See the longitude 0.125 is selected
ds.sel(longitude = 0.0, method = "nearest")

In [ ]:
# Interpolating to your point (e.g. London 51.51°N, 0.13°W): See the coordinates are the precise point you want.
# By default, a linear interpolation is performed. 
ds.interp(latitude = 51.51, longitude = 0.13)

#### Slices: subset a range of values
Sometimes, you want to subset over a range of coordinates. In that case, you can use `.sel()` with `slice`.

**NB: The order of the boundaries in `slice`has to follow the order of the dimensions you are selecting on.** i.e. if latitude is stored from -90° to +90°, you will have to select from the smallest latitude to the highest (e.g. `slice(-10, 10)`, and vice versa.

In [ ]:
# Select longitude between -5 and -2: See the longitude dimension is now 12 instead of 52.
ds.sel(longitude = slice(-5, -2))

In [ ]:
# Select data between 1990 and 2000
ds.sel(time = slice("1990", "2000"))
# See the time dimension is now 132 (=11 years *12 months) instead of 888.
# You can also check that the dates are from Jan. 1990 to Dec. 2000

#### Working with time coordinates
You might have noticed that in the above, time was selected using the "yr" or the "yr-month" syntax. xarray is rather flexible in time subsetting, and how you select depends on the frequency of your data. Basically, it understand that if you ask for "2002-01", you want all the data for this month, which, is our case, is only one point since the data is monthly. Had the data been daily, you would have obtained the 31 days. This is also why we can ask for "2002" and get the data for the whole year of 2002. 

Concretely, the data is stored with a very specific time stamp (for example, in our dataset, the monthly data is stored as the 15th or the 16th days of the month), and you can index with that, but it is not necessarily a good idea. 

This principle also apply to slices, which is why we were able to only specify the years in the example above.

In [ ]:
ds.sel(time = "2002-01") # Select the month of Jan. 2002 

In [ ]:
ds.sel(time = "2002-01-16") # Does the same because it that case the stamp is Jan. 16th 

In [ ]:
ds.sel(time = "2002").time # Selects the whole year of 2002

#### `.isel()`

In [ ]:
# Alternatively, if you now the index of the value you want instead of its coordinates, you may use .isel()
# e.g.: Get the first time step (see it shows values for January 1950)
ds.isel(time = 0) 

#### Your turn!

In [ ]:
# Task: Select the temperature for Oxford (51.75°N, 1.26°E) using the nearest value or the interpolation
# Do not forget, you need to select the variable, then select the coordinates
T_Oxford = ...
# Plot the time series
...

In [ ]:
# Task: Select the precipitation over Ireland (11°E to 5°E and 51°N to 56°N)
precip_Ireland = ...
# Display the object
...

#### *Advanced: Subsetting on a condition with `.where()`*
Sometimes, you want to subset data based on a condition (e.g. where Temperature is above 0°C). To do so, you can use the `.where()` method. 

In [ ]:
# Technically, you can use .where to do the same as .sel, although the interest is limited
print(ds.where(ds.time == "2002")) # is equivalent to ds.sel(time = "2002")
print(ds.where(ds.longitude == 0.125)) 
# However, note where masks the data according to your condition, it does not subset it.
# In the examples above, the dimensions have not been reduced. 
# The data point that do not satisfy the condition have been masked (i.e. set to NaNs). 
# You can see it if you plot it
ds.sel(time = "2002-01" # Select January 2002
      ).where(ds.latitude <= 54 # The condition: latitude is below 54°N
        ).rr.plot() # Plot precipitation

In [ ]:
# You can add the `drop = True` argument to crop the data and remove the part of the coordinates where it is all NaNs:
ds.sel(time = "2002-01" # Select January 2002
      ).where(ds.latitude <= 54, # The condition: latitude is below 54°N
              drop = True
        ).rr.plot() # Plot precipitation
# Notice the latitude axis has been cropped

In [ ]:
# .where is more useful where you want to subset based on the values of a variables, and not a coordinate
# For example, let us subset to where and when the monthly temperature was below 0°C
ds_freezing = ds.where(ds.tg < 0,)
ds_freezing

In [ ]:
# Plot the freezing placing in December 1950 and 2022
ds_freezing.sel(time = "1950-12").tg.plot()

In [ ]:
ds_freezing.sel(time = "2022-12").tg.plot()

In [ ]:
# List of the freezing months in Oxford
T_Oxford.where(T_Oxford < 0, drop = True).time

### Computation
You can perform many operations directly on the DataArrays

#### Arithmetics

In [ ]:
# Example: Add a constant to transform temperature to K
T_Kelvin = T_Oxford + 273.15

# Nota: When you change perform such operation, you may want to change the units in the attributes
T_Kelvin.attrs["units"] = 'K'

T_Kelvin

In [ ]:
# Task: Convert the pressure to Pa
P_Pa = ... # Convert by multiplying by 100
... # Change "units" attribute
P_Pa # Display the new object

In [ ]:
# You can also apply numpy functions
# Here is an example with the square root of pressure (note it makes no sense physically)
np.sqrt(ds.pp)

#### Statistics
You can perform all basic statistical operations directly on the DataArrays

In [ ]:
# Example: Compute the mean temperature over all dimensions
ds.tg.mean()

In [ ]:
# Task: Compute the mean temperature in Oxford
...
# Question: Is Oxford warmer or colder than the UK overall?

In [ ]:
# Task: compute the standard devation of the pressure with the std method
...

In [ ]:
# You may apply these operations only over a subset of dimensions
# Example: Compute the mean temperature over time only
ds.tg.mean("time") # Notice the DataArray now has only longitude and latitude as coordinates

In [ ]:
ds.tg.mean("time").plot() # Plot of the mean temperature

In [ ]:
# Task: Compute the mean over time of Ireland's precipitation
precip_Ireland_climatology = ... # Compute mean over time of precip
... # Display the object

In [ ]:
# Task: Plot the mean precipitation over Ireland
...

#### Areal means

With xarray, you can average over as many variable as you want. However, when computing areal means, always, ALWAYS, remember that all the cell in your data might not have the same surface! Therefore, you need to weight your average by the surface of the cell. 
For longitude-latitude data like ours, the area of each cell is proportional to the cosinus of the latitude.

Below, we show the average over latitude and longitude with a without weighting. Notice the difference between the two results. 

In [ ]:
# You can average very simply over several variables, e.g.,
ds.tg.mean(["longitude", "latitude"])

In [ ]:
# However, when you average data over space, remember to always be careful about the curvature of the Earth. 
# To account for it, and get a real areal mean, use a weighted mean
lat_rad = np.deg2rad(ds.latitude) # Get latitude in radians
ds.tg.weighted(np.cos(lat_rad)).mean(["latitude", "longitude"]) # Apply weights corresponding to the cosinus of the latitude before the mean
# Notice the values are slightly different than they were before

## Grouping
[See how much about grouping is covered in the pandas file]

In [ ]:
# To group we need a variable that contains the groups we want. Here, let us create a year variable:
ds["year"] = ds.time.dt.year
ds.year

In [ ]:
# Now we can group along year
ds.groupby("year")

In [ ]:
# and we can perform operations on these groups
print(ds.groupby("year").max()) # Maximum value per year
print(ds.groupby("year").mean()) # Yearly average

In [ ]:
# For time coordinates, there is actually a more convenient way to do it, as you can access the components of times directly (e.g. time.year, time.month)
print(ds.groupby("time.year").mean()) # Yearly average
print(ds.groupby("time.month").mean()) # Average over all months (=seasonal cycle)
print(ds.groupby("time.dayofyear").mean()) # Average over each day. 
# Note: dayofyear does not make a lot of sense hear as we are using monthly data, but it is useful to keep in mind for daily or hourly data.

In [ ]:
# Task: Compute the yearly temperature in Oxford
T_Oxford_yearly = ...
# Plot it
...

## Saving to netcdf
In some cases, you are going to want to save the result of your code. You can use the `to_netcdf()`method to do so.

In [ ]:
T_Oxford_yearly.to_netcdf("T_oxford_yearly.nc")

## Wrap-up on the two analyses

In [ ]:
# Yearly time series of temperature over Oxford
ds = xr.open_dataset(file) # Open the file
T = ds.tg # Select temperature
T_Oxford = T.interp(latitude = 51.75, longitude = -1.26,) # Interpolate the temperature to Oxford's coordinates
T_Oxford_yearly = T_Oxford.groupby("time.year").mean() # Compute the mean by year
T_Oxford_yearly.plot() #Plot

In [ ]:
# Precipitation climatology over Ireland
ds = xr.open_dataset(file) # Open the file
precip = ds.rr # Select temperature
precip_Ireland = ds.rr.sel(longitude = slice(-11,-5), latitude = slice(51,56)) # Select Ireland
precip_Ireland_climatology = precip_Ireland.mean("time") # Average over time
precip_Ireland_climatology.plot() # Plot

## Customize your xarray plots

In [ ]:
# Time series
T_Oxford_yearly.plot.line(
    color = "red", linestyle = "--", linewidth = 2, #Change line properties
    marker = "o", # Add markers at data points
    ylim = (7,12), # Change y-axis range
    figsize = [10,4] # Change figure size
)
plt.title("Yearly temperature in Oxford") # Change the default title
plt.xlabel("") # Remove x label
plt.ylabel("10m-temperature (°C)")

In [ ]:
# Maps
## pcolormesh vs. contourf
plt.figure()
precip_Ireland_climatology.plot.pcolormesh() # default plot
plt.figure()
precip_Ireland_climatology.plot.contourf()

In [ ]:
## Customization (works for pcolormesh or contourf
plt.figure()
precip_Ireland_climatology.plot.contourf(
    vmin = 0, vmax = 5, # Set value range for colors
    cmap = "YlGnBu", # change colormap
    levels = 11, # Split the colormap in 11 colors
    cbar_kwargs = dict(label = "Precipitation (mm)"), # Label the colorbar
)
plt.title("Ireland precipitation climatology over 1950-2023")
plt.xlabel("Longitude (°W)")
plt.ylabel("Latitude (°N)")

For more on maps, check out the maps notebook!